In [ ]:
X = vectors.toarray()
y = news.target

In [ ]:
from sklearn.model_selection import train_test_split
validation_size=0.3 #30% du jeu de données pour le test
testsize= 1-validation_size
seed=30
X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                                train_size=validation_size,
                                                random_state=seed,
                                                test_size=testsize)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from time import time
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier



# Specification des pipelines
# programmation à optimiser par une fonction :)
pipeline_SGDC = Pipeline([('tfidf', TfidfVectorizer()),
                    ('clf', SGDClassifier())])


parameters_SGDC = [
                    {'clf__max_iter': (5,),
                    'clf__alpha': (0.00001, 0.000001),
                    'clf__penalty': ('l2', 'elasticnet')}
                    ]

pipeline_DT = Pipeline([('tfidf', TfidfVectorizer()),
                   ('clf', DecisionTreeClassifier())])


#param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 5, 8, 10]
parameters_DT = [
    {'clf__min_samples_leaf': param_range,
        'clf__criterion': ['gini', 'entropy'],
        'clf__max_depth': param_range,
        'clf__min_samples_split': param_range[1:]}
]


pipeline_KNeighborsClassifier = Pipeline([('tfidf', TfidfVectorizer()),
                    ('clf', KNeighborsClassifier())])
parameters_KNeighborsClassifier = [
                    {'n_neighbors': list(range(1,15))},
                    {'metric': ['minkowski','euclidean','manhattan']}
                    ]
                     
pipeline_DecisionTreeClassifier = Pipeline([('tfidf', TfidfVectorizer()),
                    ('clf', DecisionTreeClassifier())])


parameters_DecisionTreeClassifier = [
                    {'max_depth': [1,2,3,4,5,6,7,8,9,10]},
                    {'criterion': ['gini', 'entropy']},
                    {'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10]}
                    ]
                                          
pipeline_SVC = Pipeline([('tfidf', TfidfVectorizer()),
                    ('clf', SVC())])


parameters_SVC = [
                   {'C': [0.001, 0.01, 0.1, 1, 10], 
                   'gamma' : [0.001, 0.01, 0.1, 1], 
                   'kernel': ['linear','rbf']}
                    ]
    

X=news.data
y=news.target


X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)

# Creation des GridSearchCV avec les pipelines spécifiques

gs_SGDC = GridSearchCV(pipeline_SGDC, 
                       parameters_SGDC, 
                       cv=3,
                       n_jobs=-1, 
                       scoring='accuracy')


gs_DT = GridSearchCV(pipeline_DT, 
                     parameters_DT, 
                     cv=3,
                     n_jobs=-1, 
                     scoring='accuracy')

gs_KNeighborsClassifier = GridSearchCV(pipeline_KNeighborsClassifier, 
                     parameters_KNeighborsClassifier, 
                     cv=3,
                     n_jobs=-1, 
                     scoring='accuracy')

gs_DecisionTreeClassifier = GridSearchCV(pipeline_DecisionTreeClassifier, 
                     parameters_DecisionTreeClassifier, 
                     cv=3,
                     n_jobs=-1, 
                     scoring='accuracy')

gs_SVC = GridSearchCV(pipeline_SVC, 
                     parameters_SVC, 
                     cv=3,
                     n_jobs=-1, 
                     scoring='accuracy')

grids = [gs_SGDC, gs_DT,gs_KNeighborsClassifier,gs_DecisionTreeClassifier,gs_SVC]
grid_dict={0:'Linear classifiers', 1:'Decision Tree',2:'DecisionTreeClassifier',3:'KNeighborsClassifier',4:'SVC'}

best_acc = 0.0
best_clf = 0.0
best_gs = ''

for idx,gs in enumerate(grids):
    print('\nClassifier: %s' % grid_dict[idx])
    t0 = time()
    gs.fit(X_train, y_train)
    print("Fit réalisé en %0.3fs" % (time() - t0))

    print('Meilleurs paramètres : %s' % gs.best_params_)

    print("Meilleur score d'accuracy sur l'entrainement: %.3f" % gs.best_score_)
    # Prediction sur le jeu de test avec les meilleurs paramètres
    t0 = time()
    result = gs.predict(X_test)
    print("Prédiction réalisée en %0.3fs" % (time() - t0))
    
    print("Score d'accuracy pour les meilleurs paramètres sur jeu de test : %.3f"  % accuracy_score(y_test, result))

    print ('\n matrice de confusion \n',confusion_matrix(y_test, result))

    print ('\n',classification_report(y_test, result))
    
    #Modele avec la meilleure accuracy sur le jeu de test
    if accuracy_score(y_test, result) > best_acc:
        best_acc = accuracy_score(y_test, result)
        best_gs = gs
        best_clf = idx
        
        
        
print('\nClassifier avec la meilleur accuracy sur le jeu de test\n',
      grid_dict[best_clf])        



In [ ]:
#Recupération du meilleur classifieur avec ses paramètres

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from time import time
from sklearn.metrics import classification_report


#Recupération des données pour l'exemple
#et partir proprement
categories = ['alt.atheism', 'talk.religion.misc',
              'rec.sport.hockey','comp.graphics', 'sci.space']

news = fetch_20newsgroups(subset='all',
                          categories=categories)





pipeline = Pipeline([('vect', TfidfVectorizer(preprocessor=clean_text)),
                ('clf', SGDClassifier(loss='hinge', 
                                      penalty='l2',
                                      alpha=1e-05, 
                                      random_state=42, 
                                      max_iter=5, 
                                      tol=None)),
               ])




X=news.data
y=news.target


X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)


t0 = time()
print ("Lancement du fit \n")
pipeline.fit(X_train, y_train)
print("Fit réalisé en %0.3fs" % (time() - t0))

t0 = time()
print ("Lancement de la prédiction \n")
result = pipeline.predict(X_test)
print("Prédiction réalisée en %0.3fs" % (time() - t0))

print('\n accuracy:',accuracy_score(result, y_test),'\n')

conf = confusion_matrix(y_test, result)
print ('\n matrice de confusion \n',conf)



print ('\n',classification_report(y_test, result))

print("\nSauvegarde du pipeline grid search") 
filename = 'thebestone.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

